In [1]:
!pip install langchain openai


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import json
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
qa_gen_json_file_path = "q_and_a_gen.json"

In [4]:
with open(qa_gen_json_file_path, "r") as qa_json_file:
    qa_json = json.load(qa_json_file)

In [5]:
qa_data = qa_json["data"]

In [6]:
from langchain.prompts import PromptTemplate

template = "Below are two answers from questions about a game prototyping framework called Pronto that is build on top of the Godot Game Engine. The first answer is correct. The second answer should be compared against first one and checked if it answers the question. To evaluate the result, please simply reply \"Yes\" or \"No\" whether the question was answered or not.\n\nHere is the question: {question}\n\nHere is the first answer: {answer1}\n\nHere is the second answer: {answer2}"

prompt = PromptTemplate(template=template, input_variables=["question", "answer1", "answer2"])

In [7]:
from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4")

In [8]:
from langchain.chains import LLMChain

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [9]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredMarkdownLoader

docs_loader = DirectoryLoader("pronto-docs", glob="**/*.md", loader_cls=UnstructuredMarkdownLoader)
docs = docs_loader.load()

In [10]:
from langchain_community.chat_models import ChatOpenAI

chat_llm = ChatOpenAI(temperature=0)

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language
from langchain_community.vectorstores.chroma import Chroma
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_community.embeddings import OpenAIEmbeddings

parent_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.MARKDOWN, chunk_size=400, chunk_overlap=0)
child_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.MARKDOWN, chunk_size=100, chunk_overlap=0)

pdlg_vectorstore = Chroma(
    collection_name="split_parents", embedding_function=OpenAIEmbeddings()
)
pdlg_store = InMemoryStore()
pdlg_retriever = ParentDocumentRetriever(
    vectorstore=pdlg_vectorstore,
    docstore=pdlg_store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

pdlg_retriever.add_documents(docs)

In [12]:
from langchain.retrievers import MultiQueryRetriever

pdlg_mq_retriever = MultiQueryRetriever.from_llm(
    retriever=pdlg_retriever, llm=chat_llm
)

In [13]:
from langchain.retrievers import EnsembleRetriever

ensemble_retriever = EnsembleRetriever(
    retrievers=[pdlg_mq_retriever, pdlg_retriever], weights=[0.5, 0.5]
)

In [14]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(
    llm=llm, memory_key="chat_history", return_messages=True
)

In [15]:
from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(chat_llm, retriever=ensemble_retriever, memory=memory)

In [16]:
results = {}
for data in qa_data:
    question = data["question"]
    answer1 = data["ai_answer"]
    answer2 = qa(question)
    result = llm_chain.run({
        "question": question,
        "answer1": answer1,
        "answer2": answer2
    })
    results[question] = result
    print(question, result + "/5")

How do I connect two behaviors? Yes/5
How do I adjust the properties of a PlatformerControllerBehavior? Yes/5
How can I damage a player? Yes/5
What does an AlwaysBehavior do? Yes/5
How to change the sprite of a PlaceholderBehavior? Yes/5


KeyboardInterrupt: 

In [17]:
results

{'How do I connect two behaviors?': 'Yes',
 'How do I adjust the properties of a PlatformerControllerBehavior?': 'Yes',
 'How can I damage a player?': 'Yes',
 'What does an AlwaysBehavior do?': 'Yes',
 'How to change the sprite of a PlaceholderBehavior?': 'Yes'}

In [18]:
question = qa_data[2]["question"]
answer1 = qa_data[2]["ai_answer"]
answer2 = qa(question)["answer"]
result = llm_chain.run({
    "question": question,
    "answer1": answer1,
    "answer2": answer2
})

In [19]:
result

'Yes'

In [23]:
answer2

'To damage a player in Godot, you can follow these steps:\n\n1. Create a script for the player character or the object that will cause the damage.\n\n2. Inside the script, define a function or method that will handle the damage logic. For example, you can name it "takeDamage".\n\n3. In the "takeDamage" function, you can reduce the player\'s health or apply any other desired effect. You can access the player\'s health variable or any other relevant properties within the function.\n\n4. Call the "takeDamage" function whenever you want to damage the player. This can be triggered by collisions, enemy attacks, or any other game event.\n\nHere\'s an example of what the code might look like:\n\n```gd\nextends KinematicBody2D\n\nvar health = 100\n\nfunc takeDamage(damageAmount):\n    health -= damageAmount\n    if health <= 0:\n        # Player is defeated, handle game over logic here\n\n# Example usage:\n# Call takeDamage function with a damage amount of 20\ntakeDamage(20)\n```\n\nRemember to

In [27]:
answer1

'To damage a player, you can use the `damage` method provided by the `HealthBarBehavior`. This method takes an `amount` parameter and reduces the current health value by that amount.'

In [25]:
ensemble_retriever.get_relevant_documents("How can I damage my player?")

[Document(page_content='Searches for nodes in the scene and emits signals for results. Properties allow to filter, sort, and limit results. Can be used for tasks such as destroying all enemies in a certain radius, infecting a random player, or finding the nearest health pack. \n \n SceneRootBehavior', metadata={'source': 'pronto-docs/README.md'}),
 Document(page_content='PlatformerControllerBehavior\n\nExtends: Behavior\n\nDescription\n\nConstants Descriptions\n\nPlayer\n\ngdscript\nenum Player{Player_1 = 0, Player_2 = 1, Player_3 = 2}\n\nDefines the available controls\n\nProperty Descriptions\n\nplayer\n\ngdscript\n@export var player: PlatformerControllerBehavior.Player = 0', metadata={'source': 'pronto-docs/PlatformerControllerBehavior.md'}),
 Document(page_content='Before you can deploy your game check the following:\n\nMake your game the main scene of the godot project.\n\nMake sure that your game and all assets you use are inside your prototype folder (e.g. prototypes/game-myProto

In [29]:
qa("How can I damage a player using the pronto framework?")

{'question': 'How can I damage a player using the pronto framework?',
 'chat_history': [SystemMessage(content='The human inquires about various behaviors in Godot. The AI explains that behaviors can be connected using connections, which link signals from one behavior to another. The AI provides a guide on creating these connections, emphasizing the importance of designing easily connectable behaviors. The AI also explains how to adjust the properties of a PlatformerControllerBehavior by accessing the "player" property. When asked how to damage a player, the AI suggests creating a script for the player character, defining a function for damage logic, applying the desired effect within this function, and calling this function when needed. The AI provides an example code to demonstrate this. It also explains the function of an AlwaysBehavior, which triggers every frame and listens to the signals always and physics_always. The AI tells how to change the sprite of a PlaceholderBehavior usin

In [38]:
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import initialize_agent, AgentType

tools = [
    create_retriever_tool(
        retriever=ensemble_retriever,
        name="Intermediate Answer",
        description="useful for when you need to search the pronto documentation",
    )
]

agent = initialize_agent(tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True, handle_parsing_errors=True)

In [40]:
agent.run("How can I damage a player?")



> Entering new AgentExecutor chain...
Yes.
Follow up: In which game or sport are you referring to?
Intermediate answer: [Document(page_content='gdscript\n@export var sprite_library: Texture2D\n\nSetter: @sprite_library_setter\n\nThe sprite library is desribed in [class SpriteInspector] as well as "addons/pronto/helpers/sprite_window.tscn". Search through a library of textures to choose your sprite.\n\noutline_visible\n\ngdscript\n@export var outline_visible: bool = false\n\nSetter: @outline_visible_setter', metadata={'source': 'pronto-docs/PlaceholderBehavior.md'}), Document(page_content='Before you can deploy your game check the following:\n\nMake your game the main scene of the godot project.\n\nMake sure that your game and all assets you use are inside your prototype folder (e.g. prototypes/game-myPrototype).\n\nMake sure that your branch-name starts with game- and is named identical to your folder name (e.g. game-myPrototype)', metadata={'source': 'pronto-docs/README.md'}), Docum

"In a Godot game, one way to damage a player could be to create a script where the player's health decreases when certain events occur, such as colliding with an enemy or obstacle. This can be achieved using Godot's scripting language, GD Script."